In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [54]:
# Load the data
application_test = pd.read_csv('./HomeCreditCSV/application_test.csv')
application_train = pd.read_csv('./HomeCreditCSV/application_train.csv')

**Data preprocessing**

In [55]:
def split_feature_type(data):
    categorical_list = []
    discrete_list = []
    continuos_list = []
    for i in data.columns.tolist():
        if data[i].dtype == 'object':
            categorical_list.append(i)
        elif data[i].nunique() < 20:
            discrete_list.append(i)
        else:
            continuos_list.append(i)
    return categorical_list, discrete_list, continuos_list

def label_encoder(data):
    categorical_cols = data.select_dtypes(include='object').columns
    for i in categorical_cols:
        encoder = LabelEncoder()
        data[i] = encoder.fit_transform(data[i])

In [56]:
# Imputing missing data by SimpleImputer 
categorical_list, discrete_list, continuos_list = split_feature_type(application_train)
discrete_list.remove('TARGET')

application_train[categorical_list] = SimpleImputer(
    strategy='most_frequent').fit_transform(
    application_train[categorical_list])

application_train[discrete_list] = SimpleImputer(
    strategy='most_frequent').fit_transform(
    application_train[discrete_list])

application_train[continuos_list] = SimpleImputer(
    strategy='median').fit_transform(
    application_train[continuos_list])

application_test[categorical_list] = SimpleImputer(
    strategy='most_frequent').fit_transform(
    application_test[categorical_list])

application_test[discrete_list] = SimpleImputer(
    strategy='most_frequent').fit_transform(
    application_test[discrete_list])

application_test[continuos_list] = SimpleImputer(
    strategy='median').fit_transform(
    application_test[continuos_list])

label_encoder(application_train)
label_encoder(application_test)

In [57]:
X_train = application_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
y_train = application_train.TARGET
X_pred = application_test.drop(['SK_ID_CURR'], axis=1)
y_pred = np.zeros(X_pred.shape[0])
folds = KFold(n_splits=5)
validation_pred = np.zeros(X_train.shape[0])

# Problem 1

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
for train_index, validation_index in folds.split(X_train):
    train_x, train_y = X_train.iloc[train_index], y_train[train_index]
    validation_x, validation_y = X_train.iloc[validation_index], y_train[validation_index]
    
    model_sk = lgb.LGBMClassifier(max_depth=7, learning_rate=0.05, min_split_gain=0.01, 
                                  early_stopping_rounds = 100, n_estimators= 2000,
                                  min_child_weight=40, num_leaves=24)
    
    
    model_sk.fit(train_x, train_y, early_stopping_rounds=100, 
                 eval_set = [(validation_x, validation_y)], eval_metric = 'auc', verbose = 200)
    # Estimation of validation set
    validation_pred[validation_index] = model_sk.predict(validation_x, num_iteration=model_sk.best_iteration_)
    print('AUC score: {}'.format(roc_auc_score(validation_y, validation_pred[validation_index])))

[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.759242	valid_0's binary_logloss: 0.244332
[400]	valid_0's auc: 0.761539	valid_0's binary_logloss: 0.243642
Early stopping, best iteration is:
[463]	valid_0's auc: 0.761649	valid_0's binary_logloss: 0.243592
AUC score: 0.5102507661362059
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.759564	valid_0's binary_logloss: 0.249955
[400]	valid_0's auc: 0.761377	valid_0's binary_logloss: 0.249382
Early stopping, best iteration is:
[494]	valid_0's auc: 0.761716	valid_0's binary_logloss: 0.249285
AUC score: 0.5128723337275438
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 wi

# Problem 2

In [8]:
from sklearn.model_selection import GridSearchCV
# Create params list to search
modelParams = {
    'num_leaves': [12,16,20],
    'min_split_gain': [0.01,0.02],
}
model = lgb.LGBMClassifier(max_depth=7, learning_rate=0.05, min_split_gain=0.01,
                           n_estimators= 2000, min_child_weight=40, num_leaves=24)
grid = GridSearchCV(model, modelParams, verbose=0, cv=5, n_jobs=2)
grid.fit(X_train,y_train)
print(grid.best_params_)
print(grid.best_score_)

{'min_split_gain': 0.02, 'num_leaves': 12}
0.9194727994176329


In [9]:
grid.best_estimator_

LGBMClassifier(learning_rate=0.05, max_depth=7, min_child_weight=40,
               min_split_gain=0.02, n_estimators=2000, num_leaves=12)

# Problem 3

[[Parameter tuning with Gradient Boosting Machine]](https://www.kaggle.com/willkoehrsen/intro-to-model-tuning-grid-and-random-search)<br>
 - In my implemetation of learning and verification, I use **LightGBMClassifer** which is an API for Gradient Boosting Machine for skcitlearn library. The published notebook use the Gradient Boosting Machine implementation instead for greater accuracy and speed.<br>
 - This notebook work with a subset of the data consisting of 10000 rows since Hyperparameter tuning is extremely computationally expensive and working with the full dataset in a Kaggle Kernel would not be feasible for more than a few search iterations.<br>
 - In this notebook, Grid Search and Random Search are two of approaches used for hyperparameters tuning.<br>
[[Bayesian optimization for LightGBM]](https://www.kaggle.com/sz8416/simple-bayesian-optimization-for-lightgbm)
 <br>
 - This notebook use **Bayesian Optimization** technique for hyperparameters tuning
 - The **Bayesian Optimization** is an optimization technique based on **Bayes theorem** which direct the search in order to find the minimum or maximum of an objective function.

# Problem 4

After doing some researches on both papers, I have decided to choose **BayesianOptimization** for tuning hyperparameters while still use **LightGBM** as the model for prediction.

In [50]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

For doing optimization with Bayesian model, we have to pass in an objective function in order for the Optimizer to find the greatest set of hyperparameters. <code>lgb_eval</code> is the objective function in this case.
<br>
**Cross Validation** can also be performed by inside this function using **Light GBM** API. This will return the best splits which has the greatest AUC score.

In [66]:
train_set = lgb.Dataset(data = X_train, label = y_train)

def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, 
             lambda_l1, lambda_l2, min_split_gain, min_child_weight):
    hyperParameters = {
        'application': 'binary',
        'num_iterations': 4000, 
        'learning_rate': 0.05, 
        'early_stopping_round': 100, 
        'metric':'auc',
        'num_leaves' : round(num_leaves),
        'feature_fraction': max(min(feature_fraction, 1), 0),
        'bagging_fraction': max(min(bagging_fraction, 1), 0),
        'max_depth' : round(max_depth),
        'lambda_l1' : max(lambda_l1, 0),
        'lambda_l2' : max(lambda_l2, 0),
        'min_split_gain' : min_split_gain,
        'min_child_weight' : min_child_weight
    }
    
    cv_result = lgb.cv(hyperParameters, train_set, nfold=5, 
                       seed=42, stratified=True, verbose_eval =200, metrics=['auc'])
    return max(cv_result['auc-mean'])

We creeate a **BayesianOptimizer** which takes the above function as objective function and range of parameters are specified in types of a dictionary.

In [67]:
lgbOptimizer = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                        'feature_fraction': (0.1, 0.9),
                                        'bagging_fraction': (0.8, 1),
                                        'max_depth': (5, 8.99),
                                        'lambda_l1': (0, 5),
                                        'lambda_l2': (0, 3),
                                        'min_split_gain': (0.001, 0.1),
                                        'min_child_weight': (5, 50)}, random_state=0)

# Doing optimization

In [68]:
lgbOptimizer.maximize(init_points=10, n_iter=20)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.202137 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11287
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 116
[LightGBM] [Info] Number of positive: 19860, number of negative: 226149
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.219336 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11287
[LightGBM] [Info] Number of data points in the train set: 246009, number of used features: 116
[LightGBM] [Info] Numb

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	cv_agg's auc: 0.758871 + 0.004577
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

After doing maximization, I find the set of hyperparameters which make the objective function reach the maxima and use this set for constructing the prediction model, **LGBMClassifier**. Output the result and submit it to kaggle.

In [89]:
opt_params = lgbOptimizer.max['params']
opt_params['num_leaves'] = int(round(opt_params['num_leaves']))
opt_params['max_depth'] = int(round(opt_params['max_depth']))
model = lgb.LGBMClassifier(**opt_params)
model.fit(X_train, y_train)
                        
# Predictions on the test data
preds = model.predict_proba(X_pred)[:, 1]

[LightGBM] [Warning] lambda_l1 is set=2.4229273913750986, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.4229273913750986
[LightGBM] [Warning] feature_fraction is set=0.41822836817761133, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.41822836817761133
[LightGBM] [Warning] lambda_l2 is set=1.168072817062098, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.168072817062098
[LightGBM] [Warning] bagging_fraction is set=0.8780419604530754, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8780419604530754


# Problem 5

In [90]:
submission = pd.DataFrame({'SK_ID_CURR': np.int32(application_test.SK_ID_CURR)
                           , 'TARGET': preds})
submission.to_csv('tuning_submission.csv', index = False)

![title](img/pic2.png)

The reuslt of my submission this time is much better compared to that of Class Assignment in Week 4. During doing problem 4, I have grasped some of the ideas:
 - Bayesian Optimization works very well with huge dataset while GridSearch can not perform effectively when dealing with huge set of hyperparameters and dataset.
 - Cross Validation is a wonderful technique for evaluating how good a model perform on the train set as well as preventing overfitting problems.